In [2]:
import pandas as pd

## The Data Class

This class is specific for downloading different collaborative-filtering datasets, and building new ways of simulating user ratings for the data used in this research.

In [3]:
class Data:
    
    '''
    The datasets for collaborative filtering must be:
        - The dataframe containing the ratings. 
        - It must have three columns, corresponding to the user (raw) ids, 
          the item (raw) ids, and the ratings, in this order.   
    '''
    
    def __init__(self):
        
        '''
        The databases (ml_100k, ml_1m and jester) are built-in the surprise package for
        collaborative-filtering
        '''
        
        self.available_databases=['ml_100k', 'ml_1m','jester', 'lda_topics', 'lda_rankings', 'uniform']

    def show_available_databases(self):
        print('The avaliable database are:')
        for i,database in enumerate(self.available_databases):
            print(str(i)+': '+database)            
        
    def read_data(self,database_name):
        self.database_name=database_name
        self.the_data_reader= getattr(self, 'read_'+database_name.lower())
        self.the_data_reader()   

    def read_ml_100k(self):
        
        '''
        Please search the surprise package for the documentation of this dataset.
        https://grouplens.org/datasets/movielens/
        '''
        
        from surprise import Dataset
        data = Dataset.load_builtin('ml-100k')
        self.df = pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
        self.df.drop(columns=['timestamp'],inplace=True)
        self.df.rename({'user_id':'userID','item_id':'itemID'},axis=1,inplace=True)

    def read_ml_1m(self):
        
        '''
        Please search the surprise package for the documentation of this dataset.
        https://grouplens.org/datasets/movielens/
        '''
        
        from surprise import Dataset
        data = Dataset.load_builtin('ml-1m')
        self.df = pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
        self.df.drop(columns=['timestamp'],inplace=True)
        self.df.rename({'user_id':'userID','item_id':'itemID'},axis=1,inplace=True)

    def read_jester(self):
        
        '''
        Please search the surprise package for the documentation of this dataset.
        https://eigentaste.berkeley.edu/dataset/
        '''
        
        from surprise import Dataset
        data = Dataset.load_builtin('jester')
        self.df = pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
        self.df.drop(columns=['timestamp'],inplace=True)
        self.df.rename({'user_id':'userID','item_id':'itemID'},axis=1,inplace=True)
        
    def read_uniform(self):
        
        '''
        Hyperparameters -
        `n_users` : number of simulated users in the database;
        `n_ratings` : number of simulated rating events in the database.
        
        This is a fictional dataset based in the choice of an uniformly distributed random rating 
        (from 1 to 5) for one of the simulated users of the recommender-system that is being designed in
        this research project.
        '''
        n_users = 20
        n_ratings = 10000
        
        import random
        
        opo = pd.read_csv('../oportunidades.csv')
        df = [(random.randrange(n_users), random.randrange(len(opo)), random.randrange(1,5)) for i in range(n_ratings)]
        self.df = pd.DataFrame(df, columns = ['userID', 'itemID', 'rating'])
        
    def read_lda_topics(self):
        
        '''
        Hyperparameters -
        `n_users` : number of simulated users in the database;
        `n_ratings` : number of simulated rating events in the database.
        
        This first LDA based dataset builds a model with K = `n_users` topics. LDA topics
        are used as proxies for simulated users with different clusters of interest. At first
        a random opportunity is chosen, than the amount of a randomly chosen topic inside the description
        is multiplied by five. The ceiling operation of this result is the rating that the fictional user
        will give to that opportunity.
        Because the amount of each topic predicted by the model is disollved among various topics,
        it is very rare to find an opportunity that has a higher LDA value. The consequence is that this dataset
        has really low volatility and the major part of ratings are equal to 1.
        '''
        
        n_users = 20
        n_ratings = 10000
        
        import gensim
        import random
        import math
        
        opo = pd.read_csv('../oportunidades.csv')
        try:
            lda_model = gensim.models.ldamodel.LdaModel.load(f'models/lda_model{n_users}.model')
        except:
            import generate_users
            generate_users.gen_model(n_users)
            lda_model = gensim.models.ldamodel.LdaModel.load(f'models/lda_model{n_users}.model')

        df = []
        for i in range(n_ratings):
            opo_n = random.randrange(len(opo))
            txt = opo.loc[opo_n,'opo_texto']
            opo_bow = lda_model.id2word.doc2bow(txt.split())
            topics = lda_model.get_document_topics(opo_bow)
            topics = {topic[0]:topic[1] for topic in topics}
            user = random.sample(topics.keys(), 1)[0]
            rating = math.ceil(topics[user]*5)
            df.append((user, opo_n, rating))

        self.df = pd.DataFrame(df, columns = ['userID', 'itemID', 'rating'])
        
    def read_lda_rankings(self):
        
        '''
        Hyperparameters -
        `n_users` : number of simulated users in the database;
        `n_ratings` : number of simulated rating events in the database.
        
        This second LDA based dataset builds a model with K = `n_users` topics. LDA topics
        are used as proxies for simulated users with different clusters of interest. At first
        a random opportunity is chosen, than the amounts of the topics are sorted, and the rating 
        value is equal to the quintile in which the topic is located.
        Because the ratings will be equal to the relative significance of that topic with respect
        to the overall opportunity description, this dataset has a considerably larger volatility
        and ratings are relatively equally occurrent.
        '''
        
        n_users = 20
        n_ratings = 10000
        
        import gensim
        import random
        import math
        
        opo = pd.read_csv('../oportunidades.csv')
        try:
            lda_model = gensim.models.ldamodel.LdaModel.load(f'models/lda_model{n_users}.model')
        except:
            import generate_users
            generate_users.gen_model(n_users)
            lda_model = gensim.models.ldamodel.LdaModel.load(f'models/lda_model{n_users}.model')
        
        df = []
        for i in range(n_ratings):
            opo_n = random.randrange(len(opo))
            txt = opo.loc[opo_n,'opo_texto']
            opo_bow = lda_model.id2word.doc2bow(txt.split())
            topics = lda_model.get_document_topics(opo_bow)
            topics = {topic[0]:topic[1] for topic in topics}

            prop = pd.DataFrame([topics], index=['prop']).T.sort_values('prop', ascending=True)
            prop['rating'] = range(1, len(prop)+1)
            prop['rating'] = prop['rating']/len(prop)
            prop['rating'] = prop['rating'].apply(lambda x: math.ceil(x*5))
            prop.reset_index(inplace=True)

            prop = prop.sample(1)

            df.append((prop['index'].values[0], opo_n, prop['rating'].values[0]))

        self.df = pd.DataFrame(df, columns = ['userID', 'itemID', 'rating'])
        

### Example:

In [4]:
data=Data()
data.read_data('lda_rankings')
data.df

,userID,itemID,rating
0,10,297,3
1,9,405,5
2,6,353,4
3,17,671,4
4,6,781,3
...,...,...,...
9995,8,96,2
9996,9,10,4
9997,1,844,5
9998,19,549,4


## The Method class

The *surprise* library provides 11 classifier models that try to predict the classification of training data based on several different *collaborative-filtering* techniques. The models provided with a brief explanation in English are mentioned below, for more information please refer to the [package documentation.](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html)

*random_pred.NormalPredictor*:
Algorithm predicting a random rating based on the distribution of the training set, which is assumed to be normal.

*baseline_only.BaselineOnly*:
Algorithm predicting the baseline estimate for given user and item.

*knns.KNNBasic*:
A basic collaborative filtering algorithm.

*knns.KNNWithMeans*:
A basic collaborative filtering algorithm, taking into account the mean ratings of each user.

*knns.KNNWithZScore*:
A basic collaborative filtering algorithm, taking into account the z-score normalization of each user.

*knns.KNNBaseline*:
A basic collaborative filtering algorithm taking into account a baseline rating.

*matrix_factorization.SVD*:
The famous SVD algorithm, as popularized by Simon Funk during the Netflix Prize.

*matrix_factorization.SVDpp*:
The SVD++ algorithm, an extension of SVD taking into account implicit ratings.

*matrix_factorization.NMF*:
A collaborative filtering algorithm based on Non-negative Matrix Factorization.

*slope_one.SlopeOne*:
A simple yet accurate collaborative filtering algorithm.

*co_clustering.CoClustering*:
A collaborative filtering algorithm based on co-clustering.

It is possible to pass a custom dataframe as an argument to this class. The dataframe in question needs to have 3 columns with the following name: ['userID', 'itemID', 'rating'].

In [5]:
class Method:
    def __init__(self,df):
        
        self.df=df
        self.available_methods=[
            'surprise.NormalPredictor',
            'surprise.BaselineOnly',
            'surprise.KNNBasic',
            'surprise.KNNWithMeans',
            'surprise.KNNWithZScore',
            'surprise.KNNBaseline',
            'surprise.SVD',
            'surprise.SVDpp',
            'surprise.NMF',
            'surprise.SlopeOne',
            'surprise.CoClustering',
        ]        
        
    def show_methods(self):
        print('The avaliable methods are:')
        for i,method in enumerate(self.available_methods):
            print(str(i)+': '+method)



    def run(self,the_method):
        self.the_method=the_method
        if(self.the_method[0:8]=='surprise'):
            self.run_surprise()
        elif(self.the_method[0:6]=='Gensim'):
            self.run_gensim()
        elif(self.the_method[0:13]=='Transformers-'):
            self.run_transformers()
        else:
            print('This method is not defined! Try another one.')

    def run_surprise(self):
        from surprise import Reader
        from surprise import Dataset
        from surprise.model_selection import train_test_split
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(self.df[['userID', 'itemID', 'rating']], reader)        
        trainset, testset = train_test_split(data, test_size=.30)
        the_method=self.the_method.replace("surprise.", "")
        eval(f"exec('from surprise import {the_method}')")
        the_algorithm=locals()[the_method]()
        the_algorithm.fit(trainset)
        self.predictions=the_algorithm.test(testset)
        list_predictions=[(uid,iid,r_ui,est) for uid,iid,r_ui,est,_ in self.predictions]        
        self.predictions_df = pd.DataFrame(list_predictions, columns =['user_id', 'item_id', 'rating','predicted_rating'])

## The Evaluator class

The *surprise* library provides 4 different methods to assess the accuracy of the ratings prediction. For further discussion on each metric please visit the [package documentation](https://surprise.readthedocs.io/en/stable/accuracy.html).

In [6]:
class Evaluator:

    def __init__(self,predictions_df):

        self.available_evaluators=['surprise.rmse','surprise.mse',
                                   'surprise.mae','surprise.fcp']
        self.predictions_df=predictions_df
        
    def show_evaluators(self):
        print('The avaliable evaluators are:')
        for i,evaluator in enumerate(self.available_evaluators):
            print(str(i)+': '+evaluator)
        


    def run(self,the_evaluator):        
        self.the_evaluator=the_evaluator
        if(self.the_evaluator[0:8]=='surprise'):
            self.run_surprise()
        else:
            print('This evaluator is not available!')

    def run_surprise(self):
        import surprise
        from surprise import accuracy
        predictions=[surprise.prediction_algorithms.predictions.Prediction(row['user_id'],row['item_id'],row['rating'],row['predicted_rating'],{}) for index,row in self.predictions_df.iterrows()]
        self.predictions=predictions
        self.the_evaluator= 'accuracy.' + self.the_evaluator.replace("surprise.", "")
        self.acc = eval(f'{self.the_evaluator}(predictions,verbose=True)')

## Experiment:

### Example

In [7]:
data=Data()
data.show_available_databases()
data.read_data('ml_100k')
method=Method(data.df)  
method.show_methods()
method.run('surprise.KNNWithMeans')
predictions_df=method.predictions_df
evaluator=Evaluator(predictions_df)
evaluator.show_evaluators()
evaluator.run('surprise.mse')

In [8]:
def model_table(label):
    
    '''
    Code that builds the table with the accuracy metrics for all rating prediction
    models built-in the surprise package. The expected return of this function
    is a pandas dataframe (11x4) corresponding to the 11 classifier models and
    4 different accuracy metrics.
    '''
    
    table = pd.DataFrame()
    
    data=Data()
    data.read_data(label)
    
    method=Method(data.df)
    
    for m in method.available_methods:
        print(m)
        method.run(m)
        predictions_df=method.predictions_df
        evaluator=Evaluator(predictions_df)
        
        metrics = []
        
        for e in evaluator.available_evaluators:
            evaluator.run(e)
            metrics.append(evaluator.acc)
            
        table = table.append(dict(zip(evaluator.available_evaluators,metrics)),ignore_index=True)
        
    table.index = [x[9:] for x in method.available_methods]
    table.columns = [x[9:].upper() for x in evaluator.available_evaluators]
            
    return table


import sys, os

sys.stdout = open(os.devnull, 'w') # Codigo para desativar os prints

uniform = model_table('uniform')  
topics = model_table('lda_topics')
ranking = model_table('lda_rankings')

sys.stdout = sys.__stdout__ # Codigo para reativar os prints

C:\Users\vitor\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\vitor\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release no

In [9]:
uniform

,RMSE,MSE,MAE,FCP
NormalPredictor,1.491671,2.225083,1.217832,0.505348
BaselineOnly,1.144140,1.309056,1.014428,0.488506
KNNBasic,1.199628,1.439108,1.023444,0.492627
KNNWithMeans,1.184012,1.401884,1.013719,0.505287
KNNWithZScore,1.212810,1.470909,1.037385,0.494890
KNNBaseline,1.203745,1.449003,1.034722,0.501463
SVD,1.177222,1.385852,1.020184,0.493068
SVDpp,1.183446,1.400544,1.011386,0.506882
NMF,1.306634,1.707292,1.093056,0.498184
SlopeOne,1.197773,1.434661,1.018241,0.495269


In [10]:
topics

,RMSE,MSE,MAE,FCP
NormalPredictor,0.634035,0.402000,0.419190,0.421735
BaselineOnly,0.413862,0.171282,0.247308,0.722834
KNNBasic,0.436119,0.190200,0.245587,0.696252
KNNWithMeans,0.385193,0.148374,0.223230,0.755692
KNNWithZScore,0.426524,0.181923,0.227123,0.762964
KNNBaseline,0.376838,0.142007,0.225771,0.760109
SVD,0.360758,0.130146,0.208096,0.757962
SVDpp,0.344421,0.118626,0.206569,0.820426
NMF,0.291942,0.085230,0.114447,0.819916
SlopeOne,0.354142,0.125416,0.198892,0.795067


In [11]:
ranking

,RMSE,MSE,MAE,FCP
NormalPredictor,1.899349,3.607528,1.561705,0.471117
BaselineOnly,1.078439,1.163031,0.865052,0.557469
KNNBasic,1.176811,1.384884,0.943879,0.673132
KNNWithMeans,0.977268,0.955054,0.728941,0.693852
KNNWithZScore,0.979363,0.959151,0.726589,0.709974
KNNBaseline,0.956509,0.914910,0.723466,0.710739
SVD,0.849302,0.721314,0.619954,0.785488
SVDpp,0.800241,0.640386,0.571461,0.812992
NMF,0.867444,0.752459,0.559050,0.789015
SlopeOne,0.956363,0.914630,0.702789,0.720230
